## Classifier/Recommender System

This is the third rendition, building on the previous system. It builds on the previous classification system by continuing with the GPT functions.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import *

#imports
import pandas as pd

In [2]:
# name

input_text = widgets.Text(
    placeholder='Your RFP here',
    layout = widgets.Layout(width='300px', height='auto')
)


# button send

button_send = widgets.Button(
                description='Find Recommendations',
                tooltip='Send',
                disabled = False,
                display='flex',
                flex_flow='column',
                align_items='stretch',
                layout = widgets.Layout(width='auto', height='40px')
    )

output = widgets.Output()

# stacked right hand side

text_1 = widgets.HTML(value="<h2>Input in your RFP here:</h2>")

In [3]:
from sentence_transformers import SentenceTransformer

def on_button_clicked(event):
    with output:
        clear_output()
        # Load a pre-trained Sentence Transformer model
        print("Downloading model data...")
        model = SentenceTransformer('bert-large-nli-mean-tokens')

        clear_output()
        print("Loading company data...")
        cbo_raw =  pd.read_csv("test1.csv")
        cbo_raw = cbo_raw.drop(["First Name", "Title", "Unnamed: 6", "Prompts"], axis = 1)
        cbo = cbo_raw.rename(columns={"Email" : "Desc", "Counties served" : "Counties", "Provider Names": "Names"})
        
        total_companies = len(cbo)
        rfp = input_text.value
        scores = []
        counter = 0

        clear_output()
        print("Calculating scores...")
        
        for content in cbo["Desc"]:
            counter += 1
            # Encode the bodies of text
            embeddings1 = model.encode(rfp, convert_to_tensor=True)
            embeddings2 = model.encode(content, convert_to_tensor=True)

            from scipy.spatial.distance import cosine

            # Calculate the cosine similarity
            similarity = 1 - cosine(embeddings1, embeddings2)
            
            #calculate percentage done
            percent = int(counter* 100/total_companies)
            clear_output()
            print("Calculating scores... (" + str(percent) + "%)")

            scores.append(similarity)

        #Display visulizations
        clear_output()
        print("Visualizing data...")
        cbo["Scores"] = scores
        cbo = cbo.sort_values("Scores", ascending=False)
        cbo_final = cbo[["Names", "Scores"]]
        clear_output()
        print(cbo_final)

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [4]:
vbox_text = widgets.VBox([text_1, input_text, vbox_result])

page = widgets.HBox([vbox_text])
display(page)